# Instruct GPT

若直接使用 GPT3，跟人想要的结果 miss-alignment

Instruct GPT 教 GPT 怎么生成人想要的结果。

![](../images/nlp/instruct.png)

一共分三步：

1. 人写答案，GPT 在这上面做微调，得到 SFT （supervised fine-tune）
2. SFT 生成一系列答案，标注人员对答案进行排序。以 SFT 为基础训练一个价值模型 RM，RM 最后的模型结构有一点不同，它会模仿人类进行打分。
3. 基于 SFT 训练一个 RL 模型，它也会根据 prompt 生成文本，把这个文本放到 RM 模型里让它打分，基于打分反馈用PPO算法更新模型。

RM 的 loss 为 pair-wise ranking loss:

$$\text{loss}(\theta) = \mathbb{E}_{(x,\ y_{w},\ y_{l})\sim D}\left[\log(\sigma(r_{\theta}(x, y_{w}) - r_{\theta}(x, y_{l})))\right]$$

其中标注人员认为 $y_{w}$ 比 $y_{l}$ 更好。

PPO 模型 maximize the following combined objective function in RL training:

$$
\begin{equation*}
\text{objective}(\phi) = \mathbb{E}_{(x, y)\sim D_{\pi_{\phi}^{RL}}}\left[r_{\theta}(x, y) - \beta\log(\pi_{\phi}^{RL}(y|x)/\pi_{\phi}^{SFT}(y|x))\right] + \gamma\mathbb{E}_{x\sim D_{pretrain}}\left[\log(\pi_{\phi}^{RL}(x))\right]
\end{equation*}
$$

其中：

* 第一项 $r_{\theta}(x, y)$ 是希望 RL 模型生成的答案在 RM 模型中分数高
* 第二项的意思是，RM 模型是基于 SFT 模型的输出训练出来的，如果 RL 模型跑得太偏的话会 RM 模型的打分可能不适用了，这里计算的是两个分布的散度，越低越好
* 第三项的意思是，RL 模型不仅要在 RM 模型中打分高，即生成人类觉得有价值的答案，也不要忘了语言模型目标导致它性能下降，这一步即 RLHF（reinforcement learning from human feedback）